# Exercise 1
## Validate CNPJ format and check digits

In [1]:
import logging
import unittest

In [2]:
def validate_CNPJ(cnpj):

    logging.basicConfig(level=logging.CRITICAL)         #can be changed from CRITICAL to ERROR for a helpful output 
                                                        #if executing locally, the parameter force = True might be necessary

    for char_to_remove in './-':                        #remove unwanted chars from the number
        cnpj = cnpj.replace(char_to_remove,'')

    if(not cnpj.isnumeric()):
        logging.error("CNPJ is not numeric")
        return False

    if(len(cnpj)!=14):
        logging.error("CNPJ size is not 14")
        return False

    cnpj_numbers    = [int(char) for char in cnpj[:12]] # get the data digits
    cnpj_validation = [int(char) for char in cnpj[12:]] # get the validation digits
    weights         = [5,4,3,2,9,8,7,6,5,4,3,2]         # list with the multiplication weights for the first validation
    checksum        = 0 
    
    for num, weight in zip(cnpj_numbers,weights):       # performs a dot product
        checksum += num * weight

    firstdigit = 11 - (checksum % 11)
    if(checksum % 11 < 2):                             # check the case where the validation digit is 0
        firstdigit = 0
    if(firstdigit != cnpj_validation[0]):
        logging.error("The first validation digit is wrong, should be " + str(firstdigit))
        return False

    weights.insert(0,6)                                 # add the extra weight for the second digit validation
    cnpj_numbers.append(cnpj_validation[0])             # add the first validation digit to the next dot product
    checksum = 0

    for num, weight in zip(cnpj_numbers,weights):
        checksum += num * weight

    seconddigit = 11 - (checksum % 11)
    if(checksum % 11 < 2):
        seconddigit = 0
    if(seconddigit != cnpj_validation[1]):
        logging.error("The second validation digit is wrong, should be " + str(seconddigit))
        return False

    return True

In [3]:
class test_validate_CNPJ(unittest.TestCase):
    
    def test_non_numeric(self):
        '''
        CNPJ string is not a number
        '''
        cnpj = "co.dex.123/4567-89"
        result = validate_CNPJ(cnpj)
        self.assertFalse(result)

    def test_wrong_size(self):
        '''
        CNPJ string has to many or too little numbers
        '''
        cnpj = "12.345.678/9101-1"
        result = validate_CNPJ(cnpj)
        self.assertFalse(result)

    def test_wrong_first_validation_digit(self):
        '''
        First validation digit of CNPJ string is wrong
        '''
        cnpj = "12345678910111"
        result = validate_CNPJ(cnpj)
        self.assertFalse(result)
        
    def test_wrong_second_validation_digit(self):
        '''
        Second validation digit of CNPJ string is wrong
        '''
        cnpj = "12.345.678/9101-41"
        result = validate_CNPJ(cnpj)
        self.assertFalse(result)

    def test_valid_cnpj_separators(self):
        '''
        Correct size and validation digits with sepparators
        '''
        cnpj = "12.345.678/9101-40"
        result = validate_CNPJ(cnpj)
        self.assertTrue(result)

    def test_valid_cnpj_no_separators(self):
        '''
        Correct size and validation digits with sepparators
        '''
        cnpj = "12345678910140"
        result = validate_CNPJ(cnpj)
        self.assertTrue(result)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False) 

......
----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK
